In [1]:
esHost = 'http://es01:9200'

Musí být stejný jako jmeno elasticsearche v docker-compose.yaml

In [3]:
import requests
response = requests.get(f'{esHost}/test-index')
print(response.text)

{"error":{"root_cause":[{"type":"index_not_found_exception","reason":"no such index [test-index]","resource.type":"index_or_alias","resource.id":"test-index","index_uuid":"_na_","index":"test-index"}],"type":"index_not_found_exception","reason":"no such index [test-index]","resource.type":"index_or_alias","resource.id":"test-index","index_uuid":"_na_","index":"test-index"},"status":404}


# Instalace knihovnky pro python

In [3]:
!pip install elasticsearch[async]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 6.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.3 MB/s eta 0:00:00


# Fulltextové vyhledávání v češtině

## Mazání indexu

In [4]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

{"acknowledged":true}


## Vytvoření indexu

In [7]:
doc = {
    "settings" : {
        "index" : {
            "number_of_shards" : 1, 
            "number_of_replicas" : 0 
        }
    },
    "mappings": {
        "properties": {
          "age":    { "type": "integer" },  
          "email":  { "type": "keyword"  }, 
          "name":   { "type": "text"  }     
        }
    }
}
import requests
response = requests.put(f'{esHost}/products', json=doc)
print(response.text)

{"error":{"root_cause":[{"type":"resource_already_exists_exception","reason":"index [products/snnPIAOCTEe43GU2w_2KzQ] already exists","index_uuid":"snnPIAOCTEe43GU2w_2KzQ","index":"products"}],"type":"resource_already_exists_exception","reason":"index [products/snnPIAOCTEe43GU2w_2KzQ] already exists","index_uuid":"snnPIAOCTEe43GU2w_2KzQ","index":"products"},"status":400}


## Kontrola nastavení indexu

In [8]:
import requests
response = requests.get(f'{esHost}/products/_settings')
print(response.text)

{"products":{"settings":{"index":{"routing":{"allocation":{"include":{"_tier_preference":"data_content"}}},"number_of_shards":"1","provided_name":"products","creation_date":"1684248072028","number_of_replicas":"0","uuid":"snnPIAOCTEe43GU2w_2KzQ","version":{"created":"8020099"}}}}}


## Vlastnosti indexu

In [9]:
import requests
response = requests.get(f'{esHost}/products/_mapping')
print(response.text)

{"products":{"mappings":{"properties":{"age":{"type":"integer"},"email":{"type":"keyword"},"name":{"type":"text"}}}}}


# Vkládání dokumentu do indexu

In [10]:
doc = {
    "age": 26,
    "email": "someone@somewhere.world",  
    "name": "Someone John"
}

import requests
response = requests.post(f'{esHost}/products/_doc', json=doc)
print(response.text)

{"_index":"products","_id":"M0kPJYgBm6YJMT52MFfs","_version":1,"result":"created","_shards":{"total":1,"successful":1,"failed":0},"_seq_no":0,"_primary_term":1}


In [11]:
doc = {
    "age": "26",
    "email": "some@somewhere.world",  
    "name": 4785
}

import requests
response = requests.post(f'{esHost}/products/_doc', json=doc)
print(response.text)

{"_index":"products","_id":"NEkPJYgBm6YJMT52Y1d8","_version":1,"result":"created","_shards":{"total":1,"successful":1,"failed":0},"_seq_no":1,"_primary_term":1}


## Query dle emailu

In [16]:
doc = {
   "query":{
      "match" : {
         "email":"someone@somewhere.world"
      }
   }
}

import requests
response = requests.post(f'{esHost}/products*/_search', json=doc)
print(response.text)

{"took":23,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":1,"relation":"eq"},"max_score":0.6931471,"hits":[{"_index":"products","_id":"M0kPJYgBm6YJMT52MFfs","_score":0.6931471,"_source":{"age": 26, "email": "someone@somewhere.world", "name": "Someone John"}}]}}


# Analyzer

In [22]:
doc = {
  "analyzer": "standard",
  "text": "The 2 QUICK Brown-Foxes jumped over the lazy dog's bone."
}

import requests
response = requests.post(f'{esHost}/_analyze', json=doc)
print(response.text)

{"tokens":[{"token":"the","start_offset":0,"end_offset":3,"type":"<ALPHANUM>","position":0},{"token":"2","start_offset":4,"end_offset":5,"type":"<NUM>","position":1},{"token":"quick","start_offset":6,"end_offset":11,"type":"<ALPHANUM>","position":2},{"token":"brown","start_offset":12,"end_offset":17,"type":"<ALPHANUM>","position":3},{"token":"foxes","start_offset":18,"end_offset":23,"type":"<ALPHANUM>","position":4},{"token":"jumped","start_offset":24,"end_offset":30,"type":"<ALPHANUM>","position":5},{"token":"over","start_offset":31,"end_offset":35,"type":"<ALPHANUM>","position":6},{"token":"the","start_offset":36,"end_offset":39,"type":"<ALPHANUM>","position":7},{"token":"lazy","start_offset":40,"end_offset":44,"type":"<ALPHANUM>","position":8},{"token":"dog's","start_offset":45,"end_offset":50,"type":"<ALPHANUM>","position":9},{"token":"bone","start_offset":51,"end_offset":55,"type":"<ALPHANUM>","position":10}]}


In [23]:
doc = {
  "analyzer": "standard",
  "text": "The 2 QUICK Brown-Foxes jumped over the lazy dog's bone."
}

import requests
response = requests.post(f'{esHost}/_analyze', json=doc)

txt = response.text
x = txt.split('},')
print(*x, sep = "\n")

{"tokens":[{"token":"the","start_offset":0,"end_offset":3,"type":"<ALPHANUM>","position":0
{"token":"2","start_offset":4,"end_offset":5,"type":"<NUM>","position":1
{"token":"quick","start_offset":6,"end_offset":11,"type":"<ALPHANUM>","position":2
{"token":"brown","start_offset":12,"end_offset":17,"type":"<ALPHANUM>","position":3
{"token":"foxes","start_offset":18,"end_offset":23,"type":"<ALPHANUM>","position":4
{"token":"jumped","start_offset":24,"end_offset":30,"type":"<ALPHANUM>","position":5
{"token":"over","start_offset":31,"end_offset":35,"type":"<ALPHANUM>","position":6
{"token":"the","start_offset":36,"end_offset":39,"type":"<ALPHANUM>","position":7
{"token":"lazy","start_offset":40,"end_offset":44,"type":"<ALPHANUM>","position":8
{"token":"dog's","start_offset":45,"end_offset":50,"type":"<ALPHANUM>","position":9
{"token":"bone","start_offset":51,"end_offset":55,"type":"<ALPHANUM>","position":10}]}


In [24]:
import requests
response = requests.get(f'{esHost}/products')
responseJson = response.json()
print(responseJson)

{'products': {'aliases': {}, 'mappings': {'properties': {'age': {'type': 'integer'}, 'email': {'type': 'keyword'}, 'name': {'type': 'text'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'products', 'creation_date': '1684248072028', 'number_of_replicas': '0', 'uuid': 'snnPIAOCTEe43GU2w_2KzQ', 'version': {'created': '8020099'}}}}}


In [25]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "analysis": {
      "analyzer": {
        "my_english_analyzer": {
          "type": "standard",
          "max_token_length": 5,
          "stopwords": "_english_"
        }
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [29]:
doc = {
  "analyzer": "my_english_analyzer",
  "text": "The 2 QUICK Brown-Foxes jumped over the lazy dog's bone."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['2', 'quick', 'brown', 'foxes', 'jumpe', 'd', 'over', 'lazy', "dog's", 'bone']


# Elasticsearch a fulltextové vyhledávání v češtině



## Předdefinovaný český analyzér

slova jsou převedena na malá písmena a ořezána o koncovky

In [2]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "index": {
      "number_of_shards": "1",
      "number_of_replicas": "0",
      "analysis": {
        "analyzer": {
          "czech": {
            "type": "czech"
          }
        }
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


## Kontrola

In [4]:
doc = {
  "analyzer": "czech",
  "text": "Jahody čerstvé - ve vaničce."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['jahod', 'čerstv', 'vaničk']


## Převod velkých písmen na malá

Vytváříme **custom** analyzér s názvem **czech**

tokenizer: **standard**

    filter": ["lowercase"]

In [5]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "index": {
      "number_of_shards": "1",
      "number_of_replicas": "0",
      "analysis": {
        "analyzer": {
          "czech": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": ["lowercase"]
          }
        }
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


## Kontrola

In [7]:
doc = {
  "analyzer": "czech",
  "text": "Jahody čerstvé - Ve Vaničce."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['jahody', 'čerstvé', 've', 'vaničce']


## Odstranění diakritiky

Pro odstranění diakritiky lze použít filer **asciifolding**, který nabízí elasticsearch, který převádí NE-ascii znaky na acii znaky (č->c, ř->r)
Pokud bychom chtěli použít něco zajímavějšího, tak lze použít **icu-plugin**, který je instalovaný pomocí dockerfilu (například umírozenat spojení písmen c a h jako ch)

tokenizer: **standard**

filter: **asciifolding, lowercase**

In [8]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "index": {
      "number_of_shards": "1",
      "number_of_replicas": "0",
      "analysis": {
        "analyzer": {
          "czech": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": ["asciifolding", "lowercase"]
          }
        }
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


## Kontrola

In [10]:
doc = {
  "analyzer": "czech",
  "text": "Jahody čerstvé - Ve Vaničce."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['jahody', 'cerstve', 've', 'vanicce']


## Tvarosolví
### Algoritmická stemantizace

elasticsearch nabízí český stemmer standartně k dispozici, je jen třeba ho přidat do nastavení jako filtr.

Využívá pravidel daného jazyka ktetré definu algoritmus, je velmi rychlý převod, ale mohou vznikat nějaké chyby

>do **analyzer** přidáme sekci **filter**
    
    "filter": {
          "czech_stemmer": {
            "type": "stemmer",
            "name": "czech"
          }

In [11]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "index": {
      "number_of_shards": "1",
      "number_of_replicas": "0",
      "analysis": {
        "analyzer": {
          "czech": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "czech_stemmer", 
              "asciifolding", 
              "lowercase"
            ]
          }
        },
        "filter": {
          "czech_stemmer": {
            "type": "stemmer",
            "name": "czech"
          }
        }
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [12]:
doc = {
  "analyzer": "czech",
  "text": "Jahody čerstvé - Ve Vaničce."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['jahod', 'cerstv', 've', 'vanick']


## Stematizace pomocí slovníku

Lze dosáhnout přesnějšího převodu slov, ES disponuje filtrem hunspell, který umí využívat volně dostupných slovníků (balík open office)

hunspell (slovník) máme stažený v repozitáři a dále ho importujeme pomocí dockercompose

PRO DOPŘESNĚNÍ - jako odpověd vznikne 2x jahoda, jelikož slovník vytvořil 2 slova (Jahoda a jahoda), které se teprve poté převedla na malá písmena (pro odstranění by bylo třeba obrátit pořadí)

>Je třeba přidat filter

    "filter": [
              "czech_hunspell",
              "asciifolding",
              "lowercase"
            ]
          }
        },
        "filter": {
          "czech_hunspell": {
            "type": "hunspell",
            "locale": "cs_CZ"
          }

In [13]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "index": {
      "number_of_shards": "1",
      "number_of_replicas": "0",
      "analysis": {
        "analyzer": {
          "czech": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "czech_hunspell",
              "asciifolding",
              "lowercase"
            ]
          }
        },
        "filter": {
          "czech_hunspell": {
            "type": "hunspell",
            "locale": "cs_CZ"
          }
        }
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [14]:
doc = {
  "analyzer": "czech",
  "text": "Jahody čerstvé - Ve Vaničce."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['jahoda', 'jahoda', 'cerstvy', 've', 'vanicka']


## Odstranění nevýznamných slov (stop slova)

elastic search disponuje sadou pro češtinu v rámci filtru **stop**

lze použít filre **lenght** a filtrovat podle jejich délky

Při analýze vyhledávaného textu lze filtrovat duplicitní slova, jako ve výše uvedeném případě. Tedy duplicitní slova lze filtrovat pomocí filtru **unique** 

    "filter": {
          "czech_stop": {
            "type": "stop",
            "stopwords": ["že", "_czech_"]
          },
          "czech_length": {
            "type": "length",
            "min": 2
          },
          "czech_unique": {
            "type": "unique",
            "only_on_same_position": true
          }

In [15]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "index": {
      "number_of_shards": "1",
      "number_of_replicas": "0",
      "analysis": {
        "analyzer": {
          "czech": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "czech_stop",
              "czech_length",
              "czech_unique"
            ]
          }
        },
        "filter": {
          "czech_stop": {
            "type": "stop",
            "stopwords": ["že", "_czech_"]
          },
          "czech_length": {
            "type": "length",
            "min": 2
          },
          "czech_unique": {
            "type": "unique",
            "only_on_same_position": "true"
          }
        }
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [16]:
doc = {
  "analyzer": "czech",
  "text": "Jahody čerstvé - ve Vaničce."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['Jahody', 'čerstvé', 'Vaničce']


## Kompletní analyzér pro češtinu

1. Odstraníme stop slova
2. převedeme na malá písmena
3. Odstraníme diakritiku
4. Odstraníme duplicity

PRO DOPŘESNĚNÍ - mapping analyzéru je od verze ES 8.0.0. a výše třeba definovat v settings nikoliv tako:
> PUT products/_mapping/products
{
  "products": {
    "properties": {
      "title": {
        "type": "text",
        "analyzer": "czech"
      }
    }
  }
}


In [18]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "index": {
      "number_of_shards": "1",
      "number_of_replicas": "0",
      "analysis": {
        "analyzer": {
          "czech": {
            "type": "custom",
            "tokenizer": "standard",
            "filter": [
              "czech_stop",
              "czech_hunspell",
              "lowercase",
              "czech_stop",
              "icu_folding",
              "unique_on_same_position"
            ]
          }
        },
        "filter": {
          "czech_hunspell": {
            "type": "hunspell",
            "locale": "cs_CZ"
          },
          "czech_stop": {
            "type": "stop",
            "stopwords": ["že", "_czech_"]
          },
          "unique_on_same_position": {
            "type": "unique",
            "only_on_same_position": "true"
          }
        }
      }
    }
  },
  "mappings": {
    "properties": {
      "title": {
        "type": "text",
        "analyzer": "czech"
      }
    }
  }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [19]:
doc_list = [
    {"title": "Jahody čerstvé - ve vaničce"},
    {"title": "Jahoda mražená"},
    {"title": "Maliny - vanička"}
]

import requests
for index, line in enumerate(doc_list):
    print(index, line)
    response = requests.put(f'{esHost}/products/_doc/{index}', json=line)
    responseJson = response.json()
    print(responseJson)

0 {'title': 'Jahody čerstvé - ve vaničce'}
{'_index': 'products', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
1 {'title': 'Jahoda mražená'}
{'_index': 'products', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
2 {'title': 'Maliny - vanička'}
{'_index': 'products', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}


In [20]:
doc = {
  "analyzer": "czech",
  "text": "Jahody čerstvé - ve Vaničce."
}
import requests
response = requests.post(f'{esHost}/products/_analyze', json=doc)
responseJson = response.json()
txt = responseJson

saved_words = []
for token in txt['tokens']:
    saved_words.append(token['token'])

print(saved_words)

['jahoda', 'cerstvy', 'vanicka']


In [21]:
doc = {
   "query": {
    "match": {
      "title": "jahody"
    }
  }
}

import requests
import json

response = requests.post(f'{esHost}/products*/_search', json=doc)
txt = response.text

response_json = txt
response_dict = json.loads(response_json)
# Extract titles and convert to Czech language
titles = [item["_source"]["title"] for item in response_dict["hits"]["hits"]]
for title in titles:
    print(title)

Jahoda mražená
Jahody čerstvé - ve vaničce


In [22]:
doc = {
   "query": {
    "match": {
      "title": "Vanička"
    }
  }
}

import requests
response = requests.post(f'{esHost}/products*/_search', json=doc)
txt = response.text

response_json = txt
response_dict = json.loads(response_json)
# Extract titles and convert to Czech language
titles = [item["_source"]["title"] for item in response_dict["hits"]["hits"]]
for title in titles:
    print(title)

Maliny - vanička
Jahody čerstvé - ve vaničce


# Pokročilé fulltextové vyhledávání v češtině

Analyzér bude plnit tyto úkoly:
1. Vyhledává primárně podle názvu produktu bez ohledu na tvarosloví
2. Vyhledává podle názvu produktu s ohledem na tvarosloví
3. Hledá v popistu prodkutu, ale s nejniší prioritou
***
Pro demonstaraci budou použity následující produkty:

### **Produkt 1**
* název: Jablka golden 1ks
* popisek: Veškeré ovoce je prémiové kvality
***
### **Produkt 2**
* název: Jablko idared
* popisek: Kvalitní čerstvé ovoce
***
### **Produkt 3**
* název: Musli
* popisek: Křupavé musli s jablky
***

Tedy pro výraz *jablka* budeme očekávat jako první výsledek *Jablka golden 1ks* následeně *Jablko idared* a jako poslední *musli*, jelikož se v popisku produktu vyskytuje slovo *jablky*

In [23]:
import requests
response = requests.delete(f'{esHost}/products')
print(response.text)

doc = {
  "settings": {
    "number_of_shards": "1",
    "number_of_replicas": "0",
    "analysis": {
      "analyzer": {
        "czech_hunspell_stopwords": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "min_length",
            "czech_stop",
            "czech_hunspell",
            "lowercase",
            "czech_stop",
            "icu_folding",
            "unique_on_same_position"
          ]
        },
        "czech_hunspell": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "czech_hunspell",
            "lowercase",
            "icu_folding",
            "unique_on_same_position"
          ]
        },
        "czech_lowercase": {
          "type": "custom",
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "icu_folding"
          ]
        }
      },
      "filter": {
        "czech_hunspell": {
          "type": "hunspell",
          "locale": "cs_CZ"
        },
        "czech_stop": {
          "type": "stop",
          "stopwords": [
            "že",
            "_czech_"
          ]
        },
        "unique_on_same_position": {
          "type": "unique",
          "only_on_same_position": "true"
        },
        "min_length": {
          "type": "length",
          "min": 2
        }
      }
    }
  },
  "mappings": {
      "properties": {
        "title": {
          "type": "keyword",
          "fields": {
            "czech_hunspell": {
              "type": "text",
              "analyzer": "czech_hunspell"
            },
            "czech_lowercase": {
              "type": "text",
              "analyzer": "czech_lowercase"
            }
          }
        },
        "description": {
          "type": "text",
          "analyzer": "czech_hunspell_stopwords"
        }
      }
    }
}

import requests
response = requests.put(f'{esHost}/products', json=doc)
responseJson = response.json()
print(responseJson)

{"acknowledged":true}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [24]:
doc_list = [
    {"title": "Jablka golden 1 ks", "description": "Veškeré ovoce je prémiové kvality"},
    {"title": "Jablko idared", "description": "Kvalitní a čerstvé ovoce"},
    {"title": "Müsli", "description": "Křupavé müsli s jablky"}
]

import requests
for index, line in enumerate(doc_list):
    print(index, line)
    response = requests.put(f'{esHost}/products/_doc/{index}', json=line)
    responseJson = response.json()
    print(responseJson)

0 {'title': 'Jablka golden 1 ks', 'description': 'Veškeré ovoce je prémiové kvality'}
{'_index': 'products', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
1 {'title': 'Jablko idared', 'description': 'Kvalitní a čerstvé ovoce'}
{'_index': 'products', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
2 {'title': 'Müsli', 'description': 'Křupavé müsli s jablky'}
{'_index': 'products', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}


In [27]:
doc = {
   "query": {
    "multi_match": {
      "query": "idared", 
      "fields": [
        "title.czech_lowercase^2",
        "title.czech_hunspell",
        "description^0.3"
      ]
    }
  }
}

import requests
response = requests.post(f'{esHost}/products*/_search', json=doc)
txt = response.text

response_json = txt
response_dict = json.loads(response_json)s
# Extract titles and convert to Czech language
titles = [item["_source"]["title"] for item in response_dict["hits"]["hits"]]
for title in titles:
    print(title)

Jablko idared


v Query používáme *multi_match* místo *match*, a to kvůli tomu, že lze předat seznam více polí v nichž má vyhledávat (title.czech_lowercase, title.czech_hunspell, description)

Zároveň lze definovat prioritu díky ^2 a ^0.3, kde se skóre při vyhledávání násobí dannou hodnotou a na zíkladě ní je vybráno